In [1]:
# Referencias Web Scrapping

#JavaScript

#https://www.youtube.com/watch?v=vcnomT0CP0Y 
#https://medium.com/@henriquecoura_87435/web-scraping-com-python-selenium-e-javascript-faa108f95bbe

#https://www.dataquest.io/blog/web-scraping-beautifulsoup/
#https://stackoverflow.com/questions/45448994/wait-page-to-load-before-getting-data-with-requests-get-in-python-3
#https://kiwidamien.github.io/webscraping-beyond-beautifulsoup-and-selenium.html
#https://www.codementor.io/blog/python-web-scraping-63l2v9sf2q


In [2]:
#!pip install html5lib
#!pip install selenium

In [2]:
import requests
import os
import numpy as np
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from random import randint
import warnings

import unicodedata
import re

In [3]:
"""
A remoção de acentos foi baseada em uma resposta no Stack Overflow.
http://stackoverflow.com/a/517974/3464573
"""

def removerAcentosECaracteresEspeciais(palavra):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])

    # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

In [11]:
## corrigir captura de valor do condomínio
## pegar os atributos 

houses_list= []
source_urls= []
titles = []
urls= []
addresses = []
areas = []
n_rooms = []
n_suites = []
prices = []
n_bathrooms = []
n_garages = []
condo_charges = []

# Preparing the monitoring of the loop
start_time = time.time()
requests_ = 0

for search in range(2): 
       
    if search == 0:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/?pagina="+str(page)+\
                     "#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-Santana"\
                     "_de_Parnaiba-Barrios-Aldeia_da_Serra&tipos=condominio_residencial,casa_residencial,"\
                     "sobrado_residencial"
    
    if search == 1:
        source_url = "https://www.vivareal.com.br/venda/sp/santana-de-parnaiba/bairros/morada-dos-pinheiros"\
                      "-aldeia-da-serra/condominio_residencial/?__vt=vt:b&pagina="+str(page)+\
                      "#onde=BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Morada_dos_Pinheiros_Aldeia_da_Serra"\
                      ",BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Morada_das_Flores_Aldeia_da_Serra&tipos="\
                      "condominio_residencial,sobrado_residencial"
    
    r = requests.get(source_url)

    # Pause to load the page
    time.sleep(randint(10,15))

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(r.content, 'html5lib') 

    # Get total number of pages
    search_h1 = soup.find('h1') 
    total_houses = search_h1.strong.text
    total_houses = pd.to_numeric(total_houses)
    houses_pages = np.ceil((total_houses/36)).astype(int)

   
    # For every page 
    for page in range(houses_pages):
        
        if search == 0:
            source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/?pagina="+str(page+1)+\
                         "#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-Santana"\
                         "_de_Parnaiba-Barrios-Aldeia_da_Serra&tipos=condominio_residencial,casa_residencial,"\
                         "sobrado_residencial"

        if search == 1:
            source_url = "https://www.vivareal.com.br/venda/sp/santana-de-parnaiba/bairros/morada-dos-pinheiros"\
                          "-aldeia-da-serra/condominio_residencial/?__vt=vt:b&pagina="+str(page+1)+\
                          "#onde=BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Morada_dos_Pinheiros_Aldeia_da_Serra"\
                          ",BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Morada_das_Flores_Aldeia_da_Serra&tipos="\
                          "condominio_residencial,sobrado_residencial"      
        
        r = requests.get(source_url)

        print(f"Page {page} de {houses_pages}")
        print(source_url)
        
        # Pause the loop
        time.sleep(randint(10,15))

        # Monitor the requests
        requests_ += 1
        elapsed_time = time.time() - start_time
        os.system('cls')
        print('Request:{}; Frequency: {} requests/s'.format(requests_, requests_/elapsed_time))

        # Throw a warning for non-200 status codes
        if r.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests_, response.status_code))

        # Break the loop if the number of requests is greater than expected
        houses_per_page=36

        if requests_ > houses_per_page:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        soup = BeautifulSoup(r.content, 'html5lib') 
        search_h1 = soup.find('h1') 
        total_houses = search_h1.strong.text

        # Select all the 36 house containers from a single page
        houses_list = soup.find_all('div', attrs = {'class':'property-card__main-content'})

        #print(type(houses_list))
        hoses_per_page = len(houses_list)     
        house = 1    

        for house in range(houses_per_page):

            #print(f"house {house}, de houses{houses_per_page}")
            title = houses_list[house].h2.a.text
            titles.append(title)

    #         if houses_list[house].h2.a:           
    #             title = houses_list[house].h2.a.text
    #         else: title=''
    #         titles.append(title)        

            if houses_list[house].h2.a['href']:           
                url = 'www.vivareal.com.br'+houses_list[house].h2.a['href']
            else: url=''
            urls.append(url)

            if houses_list[house].h2.span:
                address = houses_list[house].h2.span.text
            else: address = ''
            addresses.append(address)         

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-area'):
                area_m2 = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-area').text
            else: 
                area_m2 = '0'
            areas.append(area_m2)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-room js-property-detail-rooms'):
                rooms = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-room js-property-detail-rooms').text
            else: 
                rooms = '0'
            n_rooms.append(rooms)        

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites'):
                suites = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites').text
            else: suites = '0'
            n_suites.append(suites)

            if houses_list[house].section.div:
                price = houses_list[house].section.div.text
            else: price = '0'
            prices.append(price)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'):
                bathrooms = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom').text
            else: bathrooms='0'
            n_bathrooms.append(bathrooms)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-garage js-property-detail-garages'):
                garages = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-garage js-property-detail-garages').text
            else: garages='0'
            n_garages.append(garages)        

            house += 1

            #x = houses_list[h].section.footer.strong.text
            #x = houses_list[h].find('div', class_ = "property-card__price-details--condo").text
            #houses_list[h].section.footer.strong.text
            #print(x)

    page += 1

Page 0 de 16
https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/?pagina=1#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Aldeia_da_Serra&tipos=condominio_residencial,casa_residencial,sobrado_residencial
Request:1; Frequency: 0.02713351386529452 requests/s
Page 1 de 16
https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/?pagina=2#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Aldeia_da_Serra&tipos=condominio_residencial,casa_residencial,sobrado_residencial
Request:2; Frequency: 0.03588070881471524 requests/s
Page 2 de 16
https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/?pagina=3#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Aldeia_da_Serra&tipos=condominio_residencial,casa_residencial,sobrado_residencial
Request:3; Frequency: 0.040671489139406575 requests/s
Page 3 de 16
ht

KeyboardInterrupt: 

In [12]:
aldeia_imoveis = pd.DataFrame({ 'title': titles,
                                'url': urls,
                                'address': addresses,
                                'area_m2': areas,
                                'rooms': n_rooms,
                                'suites': n_suites,
                                'price': prices,
                                'bathrooms': n_bathrooms,
                                'garages': n_garages
                            })
print(aldeia_imoveis.info())
aldeia_imoveis.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      144 non-null    object
 1   url        144 non-null    object
 2   address    144 non-null    object
 3   area_m2    144 non-null    object
 4   rooms      144 non-null    object
 5   suites     144 non-null    object
 6   price      144 non-null    object
 7   bathrooms  144 non-null    object
 8   garages    144 non-null    object
dtypes: object(9)
memory usage: 10.2+ KB
None


,title,url,address,area_m2,rooms,suites,price,bathrooms,garages
0,"Casa de condomínio com 4 Quartos à Venda, 40...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Antares - Aldeia da Serra, Barueri - SP",400 Área m²,4 Quartos,2 Suítes,R$ 1.750.000,5 Banheiros,4 Vagas
1,Vila Porto,www.vivareal.com.br/imoveis-lancamento/vila-po...,"Avenida Cesário de Abreu, 262 - Aldeia da Ser...",140 Área m²,-- Quarto,0,R$ 98.000,-- Banheiro,-- Vaga
2,"Casa de condomínio com 4 Quartos à Venda, 24...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Beija-Flor - Aldeia da Serra, Barueri...",241 Área m²,4 Quartos,2 Suítes,R$ 1.150.000,4 Banheiros,4 Vagas
3,"Casa com 4 Quartos à Venda, 380m²",www.vivareal.com.br/imovel/casa-4-quartos-alde...,"Estrada Doutor Yojiro Takaoka, 2165 - Aldeia ...",380 Área m²,4 Quartos,3 Suítes,R$ 1.400.000,4 Banheiros,3 Vagas
4,"Casa com 4 Quartos à Venda, 650m²",www.vivareal.com.br/imovel/casa-4-quartos-alde...,"Avenida dos Lagos - Aldeia da Serra, Barueri ...",650 Área m²,4 Quartos,4 Suítes,R$ 2.900.000,6 Banheiros,8 Vagas
5,"Casa de condomínio com 4 Quartos à Venda, 56...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Dourada - Aldeia da Serra, Barueri - SP",566 Área m²,4 Quartos,4 Suítes,R$ 1.750.000,7 Banheiros,12 Vagas
6,"Casa com 5 Quartos à Venda, 440m²",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Jagape - Aldeia da Serra, Barueri - SP",440 Área m²,5 Quartos,5 Suítes,R$ 3.800.000,7 Banheiros,3 Vagas
7,"Casa de condomínio com 3 Quartos à Venda, 25...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Sabiá - Aldeia da Serra, Barueri - SP",254 Área m²,3 Quartos,3 Suítes,R$ 1.200.000,5 Banheiros,4 Vagas
8,Saint Marcel,www.vivareal.com.br/imoveis-lancamento/saint-m...,"Rua Vereda Tropical, sn - Aldeia da Serra, Sa...",39 Área m²,2 Quartos,0,R$ 158.900,1 Banheiro,1 Vaga
9,"Casa de condomínio com 4 Quartos à Venda, 35...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Gaivota - Aldeia da Serra, Barueri - SP",350 Área m²,4 Quartos,2 Suítes,R$ 1.250.000,5 Banheiros,2 Vagas


In [13]:
#remove acentuacoes e caracteres especiais
aldeia_imoveis['area_m2']=aldeia_imoveis['area_m2'].apply([lambda x: removerAcentosECaracteresEspeciais(x)])
aldeia_imoveis['suites']=aldeia_imoveis['suites'].apply([lambda x: removerAcentosECaracteresEspeciais(x)])
aldeia_imoveis['price']=aldeia_imoveis['price'].apply([lambda x: removerAcentosECaracteresEspeciais(x)])
    
aldeia_imoveis['area_m2']= aldeia_imoveis['area_m2'].str.replace('Area m2', '').str.strip()
aldeia_imoveis['rooms']= aldeia_imoveis['rooms'].str.replace('Quartos', '').str.strip()
aldeia_imoveis['suites']= aldeia_imoveis['suites'].str.replace('Suites', '').str.strip()
aldeia_imoveis['price']= aldeia_imoveis['price'].str.replace('R', '').str.strip()
aldeia_imoveis['bathrooms']= aldeia_imoveis['bathrooms'].str.replace('Banheiros', '').str.strip()
aldeia_imoveis['bathrooms']= aldeia_imoveis['bathrooms'].str.replace('Banheiro', '').str.strip()
aldeia_imoveis['garages']= aldeia_imoveis['garages'].str.replace('Vagas', '').str.strip()
aldeia_imoveis.head(10)

,title,url,address,area_m2,rooms,suites,price,bathrooms,garages
0,"Casa de condomínio com 4 Quartos à Venda, 40...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Antares - Aldeia da Serra, Barueri - SP",400,4,2,1750000,5,4
1,Vila Porto,www.vivareal.com.br/imoveis-lancamento/vila-po...,"Avenida Cesário de Abreu, 262 - Aldeia da Ser...",140,-- Quarto,0,98000,--,-- Vaga
2,"Casa de condomínio com 4 Quartos à Venda, 24...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Beija-Flor - Aldeia da Serra, Barueri...",241,4,2,1150000,4,4
3,"Casa com 4 Quartos à Venda, 380m²",www.vivareal.com.br/imovel/casa-4-quartos-alde...,"Estrada Doutor Yojiro Takaoka, 2165 - Aldeia ...",380,4,3,1400000,4,3
4,"Casa com 4 Quartos à Venda, 650m²",www.vivareal.com.br/imovel/casa-4-quartos-alde...,"Avenida dos Lagos - Aldeia da Serra, Barueri ...",650,4,4,2900000,6,8
5,"Casa de condomínio com 4 Quartos à Venda, 56...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Dourada - Aldeia da Serra, Barueri - SP",566,4,4,1750000,7,12
6,"Casa com 5 Quartos à Venda, 440m²",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Jagape - Aldeia da Serra, Barueri - SP",440,5,5,3800000,7,3
7,"Casa de condomínio com 3 Quartos à Venda, 25...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Sabiá - Aldeia da Serra, Barueri - SP",254,3,3,1200000,5,4
8,Saint Marcel,www.vivareal.com.br/imoveis-lancamento/saint-m...,"Rua Vereda Tropical, sn - Aldeia da Serra, Sa...",39,2,0,158900,1,1 Vaga
9,"Casa de condomínio com 4 Quartos à Venda, 35...",www.vivareal.com.br/imovel/casa-de-condominio-...,"Alameda Gaivota - Aldeia da Serra, Barueri - SP",350,4,2,1250000,5,2


In [14]:
aux = aldeia_imoveis.copy()
aux =  aux[ aux['title'].str.contains('Casa') | aux['title'].str.contains('Sobrado')]

In [15]:
aux.to_csv('../output/imoveis_aldeia', index=None)